In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("labeled_sensor_data.csv")

# Check for missing values in x, y, and z columns
missing_values = df[['x', 'y', 'z']].isnull().sum()

print("Missing values in each column:")
print(missing_values)


Missing values in each column:
x    0
y    0
z    0
dtype: int64


In [4]:
duplicate_rows = df[df.duplicated()]

print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")
print("Duplicate rows (if any):")
print(duplicate_rows)


Number of duplicate rows: 0
Duplicate rows (if any):
Empty DataFrame
Columns: [directory, subject, timestamp, x, y, z, activity_type]
Index: []


In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Standardize x, y, z columns
df[['x', 'y', 'z']] = scaler.fit_transform(df[['x', 'y', 'z']])

In [6]:
# Sort by subject, activity_type, and timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', dayfirst=False, errors='coerce').dt.tz_convert('UTC')
df = df.sort_values(['subject', 'activity_type', 'timestamp'])

# Compute time differences within each subject and activity_type group
df['time_diff'] = df.groupby(['subject', 'activity_type'])['timestamp'].diff().dt.total_seconds()

# Assign a new sequence_id when time_diff > 1 second (indicating a gap)
df['sequence_id'] = (df['time_diff'] > 1.0).cumsum()

# Drop time_diff as it’s no longer needed
df = df.drop(columns=['time_diff'])

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def generate_windows(group, window_size=pd.Timedelta(seconds=2), step_size=pd.Timedelta(seconds=1)):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            windows.append(window_data)
            labels.append(group['activity_type'].iloc[0])  # All rows have the same activity_type
        current_start += step_size
    
    return windows, labels

# Process all sequences
all_windows = []
all_labels = []
for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
    windows, labels = generate_windows(group)
    all_windows.extend(windows)
    all_labels.extend(labels)

In [8]:
def extract_features(window_data):
    features = {}
    
    # Add subject
    features['subject'] = window_data['subject'].iloc[0]
    
    # Time-Domain Features
    for axis in ['x', 'y', 'z']:
        data = window_data[axis]
        features[f'{axis}_mean'] = data.mean()
        features[f'{axis}_std'] = data.std()
        features[f'{axis}_skew'] = skew(data)
        features[f'{axis}_kurt'] = kurtosis(data)
    
    # Magnitude Features
    mag = np.sqrt(window_data['x']**2 + window_data['y']**2 + window_data['z']**2)
    features['mag_mean'] = mag.mean()
    features['mag_std'] = mag.std()
    
    # Frequency-Domain Features
    for axis in ['x', 'y', 'z']:
        fft_vals = np.abs(np.fft.fft(window_data[axis]))[:len(window_data[axis])//2]
        freqs = np.fft.fftfreq(len(window_data[axis]), d=0.135)[:len(window_data[axis])//2]
        for low, high in [(0, 1), (1, 2), (2, 3), (3, 3.7)]:
            mask = (freqs >= low) & (freqs < high)
            features[f'{axis}_power_{low}-{high}'] = np.sum(fft_vals[mask]**2) if mask.any() else 0
    
    # Step Frequency
    peaks, _ = find_peaks(mag, height=mag.mean() + mag.std(), distance=3)
    window_duration = (window_data['timestamp'].max() - window_data['timestamp'].min()).total_seconds()
    features['step_frequency'] = len(peaks) / window_duration if window_duration > 0 else 0
    
    return features


# Extract features from all windows
feature_list = [extract_features(window) for window in all_windows]
features_df = pd.DataFrame(feature_list)
features_df['label'] = all_labels

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features_df['label'] = le.fit_transform(features_df['label'])

In [10]:
subjects = features_df['subject'].unique()
train_subjects = subjects[:6]
test_subjects = subjects[6:]

train_df = features_df[features_df['subject'].isin(train_subjects)]
test_df = features_df[features_df['subject'].isin(test_subjects)]


X_train = train_df.drop(columns=['label', 'subject'])
y_train = train_df['label']
X_test = test_df.drop(columns=['label', 'subject'])
y_test = test_df['label']

In [11]:
train_df.subject.value_counts(),train_subjects,test_subjects

(subject
 U3     700
 U22    526
 U4     526
 U2     406
 U21    234
 U1     232
 Name: count, dtype: int64,
 array(['U1', 'U2', 'U21', 'U22', 'U3', 'U4'], dtype=object),
 array(['U5', 'U6', 'U7'], dtype=object))

In [12]:
train_df

,subject,x_mean,x_std,x_skew,x_kurt,y_mean,y_std,y_skew,y_kurt,z_mean,...,y_power_0-1,y_power_1-2,y_power_2-3,y_power_3-3.7,z_power_0-1,z_power_1-2,z_power_2-3,z_power_3-3.7,step_frequency,label
0,U1,1.039046,0.289536,0.636500,0.186002,-2.825303,0.237446,0.851809,0.515705,-1.090639,...,1799.461384,1.752469,0.680077,0.000000,268.867212,0.738078,0.080534,0.000000,0.537057,1
1,U1,1.051399,0.191080,0.299686,3.119047,-2.291205,0.406181,-0.338864,-0.093433,-0.422957,...,5432.738400,15.490807,6.251968,2.869321,264.598168,9.064423,2.368158,0.617669,1.033592,1
2,U1,0.799902,0.335252,-0.543869,-0.810680,-1.711159,0.658912,0.085218,-0.768201,0.032903,...,5482.616175,35.517607,14.231575,5.963741,157.065633,12.131387,0.752298,1.190038,1.125492,1
3,U1,0.275722,0.324626,-0.052842,-1.212538,-0.826562,0.409405,-0.209649,-1.130426,0.447884,...,466.856616,7.286460,1.606632,1.024365,128.380347,1.003185,0.429495,0.145687,0.000000,1
4,U1,-0.082938,0.079295,-0.767694,0.276705,-0.303571,0.135254,-0.368033,-1.185725,0.425295,...,25.377995,0.326606,0.069823,0.002830,46.449367,0.140392,0.124444,0.040101,0.502513,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,U4,1.195507,0.081130,-0.019610,-0.182617,-1.101917,0.059593,-0.499873,-0.111056,-0.063556,...,273.443704,0.042913,0.063232,0.000000,0.978934,0.066375,0.048606,0.000000,1.074114,9
2620,U4,1.217650,0.068399,0.714285,-0.027585,-1.040457,0.045709,-0.052265,-0.622718,-0.081475,...,243.596134,0.077637,0.105613,0.000000,1.545319,0.088410,0.020964,0.000000,1.102536,9
2621,U4,1.238440,0.090012,-1.011424,0.825115,-1.086743,0.081806,0.011404,-1.263044,-0.081731,...,266.264064,0.078372,0.071770,0.000000,1.645587,0.028046,0.046734,0.000000,0.536481,9
2622,U4,1.242361,0.270512,0.296077,2.445001,-1.101090,0.113808,0.885632,-0.087745,-0.112327,...,274.028419,0.065500,0.034104,0.000000,3.335403,0.011512,0.111845,0.000000,1.089918,9


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.245


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing: Normalize per subject
def normalize_per_subject(df):
    scaler = StandardScaler()
    for subject in df['subject'].unique():
        mask = df['subject'] == subject
        df.loc[mask, ['x', 'y', 'z']] = scaler.fit_transform(df.loc[mask, ['x', 'y', 'z']])
    return df

# Data Augmentation: Add jitter
def augment_data(window_data):
    noise = np.random.normal(0, 0.01, window_data[['x', 'y', 'z']].shape)
    window_data[['x', 'y', 'z']] += noise
    return window_data

# Generate Windows
def generate_windows(group, window_size=pd.Timedelta(seconds=1), step_size=pd.Timedelta(seconds=0.5)):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    subjects = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            # Apply augmentation with 50% probability
            if np.random.rand() > 0.5:
                window_data = augment_data(window_data.copy())
            windows.append(window_data)
            labels.append(group['activity_type'].iloc[0])
            subjects.append(group['subject'].iloc[0])
        current_start += step_size
    
    return windows, labels, subjects

# Feature Extraction
def extract_features(window_data):
    features = {}
    
    # Time-Domain Features
    for axis in ['x', 'y', 'z']:
        data = window_data[axis]
        features[f'{axis}_mean'] = data.mean()
        features[f'{axis}_std'] = data.std()
        features[f'{axis}_skew'] = skew(data)
        features[f'{axis}_kurt'] = kurtosis(data)
        features[f'{axis}_min'] = data.min()
        features[f'{axis}_max'] = data.max()
        features[f'{axis}_range'] = data.max() - data.min()
    
    # Magnitude Features
    mag = np.sqrt(window_data['x']**2 + window_data['y']**2 + window_data['z']**2)
    features['mag_mean'] = mag.mean()
    features['mag_std'] = mag.std()
    
    # Cross-Axis Correlation
    features['xy_corr'] = window_data['x'].corr(window_data['y'])
    features['xz_corr'] = window_data['x'].corr(window_data['z'])
    features['yz_corr'] = window_data['y'].corr(window_data['z'])
    
    # Frequency-Domain Features
    for axis in ['x', 'y', 'z']:
        fft_vals = np.abs(np.fft.fft(window_data[axis]))[:len(window_data[axis])//2]
        freqs = np.fft.fftfreq(len(window_data[axis]), d=0.135)[:len(window_data[axis])//2]
        for low, high in [(0, 1), (1, 2), (2, 3), (3, 3.7)]:
            mask = (freqs >= low) & (freqs < high)
            features[f'{axis}_power_{low}-{high}'] = np.sum(fft_vals[mask]**2) if mask.any() else 0
    
    # Step Frequency
    peaks, _ = find_peaks(mag, height=mag.mean() + mag.std(), distance=3)
    window_duration = (window_data['timestamp'].max() - window_data['timestamp'].min()).total_seconds()
    features['step_frequency'] = len(peaks) / window_duration if window_duration > 0 else 0
    
    # Convert to numerical array
    feature_values = [v if not np.isnan(v) else 0 for k, v in features.items()]
    return np.array(feature_values)

# Prepare Sequences
def prepare_sequences(df, max_seq_len=50):
    sequences = []
    labels = []
    subjects = []
    max_len = 0
    for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
        windows, window_labels, window_subjects = generate_windows(group)
        if windows:
            sequence_features = [extract_features(window) for window in windows]
            seq_len = len(sequence_features)
            if seq_len > max_seq_len:
                sequence_features = sequence_features[:max_seq_len]
                seq_len = max_seq_len
            max_len = max(max_len, seq_len)
            sequences.append(np.array(sequence_features))
            labels.append(window_labels[0])
            subjects.append(window_subjects[0])
    print(f"Maximum sequence length: {max_len}")
    return sequences, labels, subjects, max_len

# Custom Dataset
class ActivityDataset(Dataset):
    def __init__(self, sequences, labels, subjects, subject_encoder):
        self.sequences = sequences
        self.labels = labels
        self.subjects = [subject_encoder.get(subj, len(subject_encoder)) for subj in subjects]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    max_len = max(seq.shape[0] for seq in sequences)
    padded_sequences = []
    masks = []
    for seq in sequences:
        pad_width = ((0, max_len - seq.shape[0]), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
        mask = np.ones(seq.shape[0], dtype=bool)
        mask = np.pad(mask, (0, max_len - seq.shape[0]), mode='constant', constant_values=False)
        masks.append(mask)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(masks, dtype=torch.bool),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(subjects, dtype=torch.long))

# Transformer Model
class ActivityTransformer(nn.Module):
    def __init__(self, num_features, num_classes, num_subjects, max_seq_len, d_model=64, num_heads=4, num_layers=1, dropout=0.3, embedding_dim=16):
        super(ActivityTransformer, self).__init__()
        self.d_model = d_model
        self.subject_embedding = nn.Embedding(num_subjects + 1, embedding_dim)
        self.input_proj = nn.Linear(num_features + embedding_dim, d_model)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_len + 1, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, dim_feedforward=256, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
    
    def forward(self, x, mask, subject_ids):
        batch_size, seq_len, num_features = x.shape
        subj_emb = self.subject_embedding(subject_ids).unsqueeze(1).expand(-1, seq_len, -1)
        x = torch.cat((x, subj_emb), dim=-1)
        x = self.input_proj(x)
        cls_token = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        pos_encoding = self.pos_encoder[:, :seq_len + 1, :]
        x += pos_encoding
        cls_mask = torch.ones(batch_size, 1, dtype=torch.bool, device=x.device)
        full_mask = torch.cat((cls_mask, mask), dim=1)
        transformer_mask = ~full_mask
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x, src_key_padding_mask=transformer_mask)
        x = x.permute(1, 0, 2)
        cls_output = x[:, 0, :]
        return self.classifier(cls_output)

# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        sequences, masks, labels, subjects = [b.to(device) for b in batch]
        optimizer.zero_grad()
        logits = model(sequences, masks, subjects)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device, le):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            sequences, masks, labels, subjects = [b.to(device) for b in batch]
            logits = model(sequences, masks, subjects)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='weighted')
    # Per-class F1
    per_class_f1 = f1_score(true, preds, average=None)
    print("Per-class F1-scores:", dict(zip(le.classes_, per_class_f1)))
    # Confusion Matrix
    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()
    return acc, f1

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
    
    def __call__(self, val_f1, model):
        score = val_f1
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


# Preprocess
df = normalize_per_subject(df)
sequences, labels, subjects, max_seq_len = prepare_sequences(df)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
subject_encoder = {subj: idx for idx, subj in enumerate(sorted(set(subjects)))}

# Split for a single fold
# train_idx = [i for i, s in enumerate(subjects) if s != 'U3']
# val_idx = [i for i, s in enumerate(subjects) if s == 'U3']
# train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
# val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

# # Datasets and Loaders
# train_dataset = ActivityDataset(*train_data, subject_encoder)
# val_dataset = ActivityDataset(*val_data, subject_encoder)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


val_subjects = ['U3', 'U8']  
train_idx = [i for i, s in enumerate(subjects) if s not in val_subjects]
val_idx = [i for i, s in enumerate(subjects) if s in val_subjects]

train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

train_dataset = ActivityDataset(*train_data, subject_encoder)
val_dataset = ActivityDataset(*val_data, subject_encoder)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


# Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = sequences[0].shape[1]  # Should be 24: (7 time + 4 freq) * 3 axes + 2 mag + 3 corr + 1 step
model = ActivityTransformer(
    num_features=num_features,
    num_classes=len(le.classes_),
    num_subjects=len(subject_encoder),
    max_seq_len=max_seq_len,
    d_model=64,
    num_heads=4,
    num_layers=1,
    dropout=0.3,
    embedding_dim=16
).to(device)
class_weights = torch.tensor([1.0 / Counter(labels_encoded)[i] for i in range(len(le.classes_))], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
early_stopping = EarlyStopping(patience=5)

# Training Loop
for epoch in range(50):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device, le)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")
    scheduler.step(train_loss)
    # early_stopping(val_f1, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Maximum sequence length: 50
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 0.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0, '8 Walk (LEFT --> Right --> Left)': 0.0}


c:\Users\rohan\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\rohan\AppData\Local\Temp\ipykernel_45908\3929876058.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 1: Loss=2.4235, Val Acc=0.0000, Val F1=0.0000
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 0.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0}


C:\Users\rohan\AppData\Local\Temp\ipykernel_45908\3929876058.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 2: Loss=2.3805, Val Acc=0.0000, Val F1=0.0000
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 0.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.0, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0, '8 Walk (LEFT --> Right --> Left)': 0.0}


...

Epoch 47: Loss=1.1348, Val Acc=0.4000, Val F1=0.3667
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.6666666666666666, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0, '8 Walk (LEFT --> Right --> Left)': 0.5}
Epoch 48: Loss=1.1085, Val Acc=0.4000, Val F1=0.3667
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.6666666666666666, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - si

C:\Users\rohan\AppData\Local\Temp\ipykernel_45908\3929876058.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 49: Loss=1.1019, Val Acc=0.4000, Val F1=0.3667
Per-class F1-scores: {'1 (FACING camera) Sit and stand': 0.0, '10 Slow walk (SHAKING hands/body, tiny step, head forward)': 1.0, '2 (FACING camera) both hands SHAKING (sitting position)': 0.0, '3 Stand up from chair - both hands with SHAKING': 0.6666666666666666, '4 (Sideway) Sit & stand': 0.0, '5 (Sideway) both hands SHAKING (sitting)': 0.0, '6 (Sideway) STAND up with - both hands SHAKING': 0.0, '7 Cool down - sitting/relax': 0.0, '8 Walk (LEFT --> Right --> Left)': 0.5}


C:\Users\rohan\AppData\Local\Temp\ipykernel_45908\3929876058.py:141: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  torch.tensor(masks, dtype=torch.bool),


Epoch 50: Loss=1.0616, Val Acc=0.4000, Val F1=0.3667


### CNN


In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import torch.optim as optim

# Assuming df is loaded with columns: 'subject', 'activity_type', 'sequence_id', 'timestamp', 'x', 'y', 'z'

# Generate windows of raw accelerometer data
def generate_windows(group, window_size=pd.Timedelta(seconds=2), step_size=pd.Timedelta(seconds=1), fixed_samples=15):
    group = group.sort_values('timestamp')
    start_time = group['timestamp'].min()
    end_time = group['timestamp'].max()
    windows = []
    labels = []
    subjects = []
    current_start = start_time
    
    while current_start + window_size <= end_time:
        window_end = current_start + window_size
        window_data = group[(group['timestamp'] >= current_start) & (group['timestamp'] < window_end)]
        if not window_data.empty:
            raw_window = window_data[['x', 'y', 'z']].values
            if len(raw_window) < fixed_samples:
                pad_width = ((0, fixed_samples - len(raw_window)), (0, 0))
                raw_window = np.pad(raw_window, pad_width, mode='constant', constant_values=0)
            elif len(raw_window) > fixed_samples:
                raw_window = raw_window[:fixed_samples]
            windows.append(raw_window)
            labels.append(group['activity_type'].iloc[0])
            subjects.append(group['subject'].iloc[0])
        current_start += step_size
    
    return windows, labels, subjects

# Collect all windows, labels, and subjects
all_windows = []
all_labels = []
all_subjects = []
for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
    windows, labels, subjects = generate_windows(group)
    all_windows.extend(windows)
    all_labels.extend(labels)
    all_subjects.extend(subjects)

# Encode activity labels
le = LabelEncoder()
all_labels_encoded = le.fit_transform(all_labels)

# Split data for leave-one-subject-out validation (example: validate on 'U3')
val_subject = 'U3'
train_idx = [i for i, s in enumerate(all_subjects) if s != val_subject]
val_idx = [i for i, s in enumerate(all_subjects) if s == val_subject]

train_windows = [all_windows[i] for i in train_idx]
train_labels = [all_labels_encoded[i] for i in train_idx]
val_windows = [all_windows[i] for i in val_idx]
val_labels = [all_labels_encoded[i] for i in val_idx]

# Define custom Dataset class
class WindowDataset(Dataset):
    def __init__(self, windows, labels, subjects):
        self.windows = windows
        self.labels = labels
        self.subjects = subjects
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        window = torch.tensor(self.windows[idx], dtype=torch.float32).transpose(0, 1)  # Shape: (3, 15)
        label = self.labels[idx]
        subject = self.subjects[idx]
        return window, label, subject

# Create DataLoaders
train_dataset = WindowDataset(train_windows, train_labels, [all_subjects[i] for i in train_idx])
val_dataset = WindowDataset(val_windows, val_labels, [all_subjects[i] for i in val_idx])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define CNN model
class ActivityCNN(nn.Module):
    def __init__(self, num_classes):
        super(ActivityCNN, self).__init__()
        self.conv1 = nn.Conv1d(3, 32, kernel_size=3)  # Input channels: 3 (x, y, z)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(64 * 2, 128)  # Adjust based on output size after conv/pool
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ActivityCNN(num_classes=len(le.classes_)).to(device)

# Compute class weights for imbalance
class_counts = Counter(train_labels)
class_weights = [1.0 / class_counts[i] for i in range(len(le.classes_))]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        windows, labels, _ = batch
        windows = windows.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(windows)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# Evaluation function
def evaluate(model, loader, device):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            windows, labels, _ = batch
            windows = windows.to(device)
            labels = labels.to(device)

            logits = model(windows)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='weighted')
    return acc, f1


# Training loop
for epoch in range(100):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")

Epoch 1: Loss=2.1925, Val Acc=0.1871, Val F1=0.1767
Epoch 2: Loss=2.1206, Val Acc=0.1443, Val F1=0.1547
Epoch 3: Loss=2.0762, Val Acc=0.1200, Val F1=0.1138
Epoch 4: Loss=2.0239, Val Acc=0.0843, Val F1=0.1037
Epoch 5: Loss=1.9787, Val Acc=0.1243, Val F1=0.0915
Epoch 6: Loss=1.9274, Val Acc=0.1286, Val F1=0.0836
Epoch 7: Loss=1.8751, Val Acc=0.1200, Val F1=0.0890
Epoch 8: Loss=1.8256, Val Acc=0.1400, Val F1=0.0946
Epoch 9: Loss=1.7914, Val Acc=0.1614, Val F1=0.1307
Epoch 10: Loss=1.7683, Val Acc=0.1143, Val F1=0.0958
Epoch 11: Loss=1.7092, Val Acc=0.1443, Val F1=0.1161
Epoch 12: Loss=1.6792, Val Acc=0.1386, Val F1=0.1319
Epoch 13: Loss=1.6518, Val Acc=0.1471, Val F1=0.1347
Epoch 14: Loss=1.6262, Val Acc=0.1414, Val F1=0.1232
Epoch 15: Loss=1.5881, Val Acc=0.1471, Val F1=0.1300
Epoch 16: Loss=1.5719, Val Acc=0.1643, Val F1=0.1511
Epoch 17: Loss=1.5343, Val Acc=0.1629, Val F1=0.1529
Epoch 18: Loss=1.5214, Val Acc=0.1429, Val F1=0.1346
Epoch 19: Loss=1.4862, Val Acc=0.1571, Val F1=0.1357
Ep

In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import torch.optim as optim

# Assuming df is a DataFrame with columns: 'subject', 'activity_type', 'sequence_id', 'timestamp', 'x', 'y', 'z'

# Prepare sequences from raw data
sequences = []
labels = []
subjects = []
max_seq_len = 300  # Maximum sequence length; adjust as needed

for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
    seq = group[['x', 'y', 'z']].values  # Raw accelerometer data
    if len(seq) > max_seq_len:
        seq = seq[:max_seq_len]  # Truncate long sequences
    sequences.append(seq)
    labels.append(activity_type)
    subjects.append(subject)

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split data (example: leave-one-subject-out with 'U3' as validation)
val_subject = 'U3'
train_idx = [i for i, s in enumerate(subjects) if s != val_subject]
val_idx = [i for i, s in enumerate(subjects) if s == val_subject]

train_sequences = [sequences[i] for i in train_idx]
train_labels = [labels_encoded[i] for i in train_idx]
val_sequences = [sequences[i] for i in val_idx]
val_labels = [labels_encoded[i] for i in val_idx]

# Custom Dataset
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels, subjects):
        self.sequences = sequences
        self.labels = labels
        self.subjects = subjects
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

# Collate function to pad sequences in each batch
def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    max_len = max(lengths)
    padded_sequences = []
    for seq in sequences:
        pad_width = ((0, max_len - len(seq)), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(lengths, dtype=torch.long),
            subjects)

# LSTM Model
class ActivityLSTM(nn.Module):
    def __init__(self, input_size=3, hidden_size=64, num_layers=2, num_classes=10, bidirectional=True):
        super(ActivityLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)
    
    def forward(self, x, lengths):
        lstm_out, _ = self.lstm(x)  # Shape: (batch_size, seq_len, hidden_size * directions)
        batch_size = x.size(0)
        indices = torch.arange(batch_size, device=x.device)
        last_outputs = lstm_out[indices, lengths - 1, :]  # Get output at last non-padded step
        logits = self.fc(last_outputs)
        return logits

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
    
    def __call__(self, val_f1, model):
        score = val_f1
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ActivityLSTM(num_classes=len(le.classes_)).to(device)

# Class weights for imbalance
class_counts = Counter(train_labels)
class_weights = [1.0 / class_counts[i] for i in range(len(le.classes_))]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=0.01)

# DataLoaders
val_subjects = ['U3', 'U8']  # Choose at least 3 subjects for validation

train_idx = [i for i, s in enumerate(subjects) if s not in val_subjects]
val_idx = [i for i, s in enumerate(subjects) if s in val_subjects]

train_data = ([sequences[i] for i in train_idx], labels_encoded[train_idx], [subjects[i] for i in train_idx])
val_data = ([sequences[i] for i in val_idx], labels_encoded[val_idx], [subjects[i] for i in val_idx])

train_dataset = ActivityDataset(*train_data, subject_encoder)
val_dataset = ActivityDataset(*val_data, subject_encoder)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        padded_sequences, labels, lengths, _ = [b.to(device) if isinstance(b, torch.Tensor) else b for b in batch]
        optimizer.zero_grad()
        logits = model(padded_sequences, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            padded_sequences, labels, lengths, _ = [b.to(device) if isinstance(b, torch.Tensor) else b for b in batch]
            logits = model(padded_sequences, lengths)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    return accuracy_score(true, preds), f1_score(true, preds, average='weighted')

# Training Loop
early_stopping = EarlyStopping(patience=5)
for epoch in range(40):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1 = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")
    early_stopping(val_f1, model)
    # if early_stopping.early_stop:
    #     print("Early stopping triggered")
    #     break

Epoch 1: Loss=2.3283, Val Acc=0.3000, Val F1=0.2300
Epoch 2: Loss=2.2104, Val Acc=0.2000, Val F1=0.0800
Epoch 3: Loss=2.1216, Val Acc=0.1000, Val F1=0.0250
Epoch 4: Loss=2.0581, Val Acc=0.1000, Val F1=0.0286
Epoch 5: Loss=1.9798, Val Acc=0.1000, Val F1=0.0222
Epoch 6: Loss=1.9545, Val Acc=0.1000, Val F1=0.0333
Epoch 7: Loss=1.9387, Val Acc=0.2000, Val F1=0.0786
Epoch 8: Loss=1.8702, Val Acc=0.2000, Val F1=0.0833
Epoch 9: Loss=1.9175, Val Acc=0.1000, Val F1=0.0500
Epoch 10: Loss=1.8443, Val Acc=0.2000, Val F1=0.0917
Epoch 11: Loss=1.8050, Val Acc=0.2000, Val F1=0.0917
Epoch 12: Loss=1.7697, Val Acc=0.1000, Val F1=0.0667
Epoch 13: Loss=1.7584, Val Acc=0.2000, Val F1=0.0952
Epoch 14: Loss=1.7504, Val Acc=0.2000, Val F1=0.0952
Epoch 15: Loss=1.7149, Val Acc=0.2000, Val F1=0.0750
Epoch 16: Loss=1.6512, Val Acc=0.1000, Val F1=0.0500
Epoch 17: Loss=1.5641, Val Acc=0.2000, Val F1=0.1000
Epoch 18: Loss=1.5370, Val Acc=0.1000, Val F1=0.1000
Epoch 19: Loss=1.5541, Val Acc=0.3000, Val F1=0.1500
Ep

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import torch.optim as optim

# Assuming df is a DataFrame with columns: 'subject', 'activity_type', 'sequence_id', 'timestamp', 'x', 'y', 'z'

# Prepare sequences from raw data
sequences = []
labels = []
subjects = []
max_seq_len = 300  # Maximum sequence length; adjust based on data

for (subject, activity_type, sequence_id), group in df.groupby(['subject', 'activity_type', 'sequence_id']):
    seq = group[['x', 'y', 'z']].values  # Raw accelerometer data
    if len(seq) > max_seq_len:
        seq = seq[:max_seq_len]  # Truncate long sequences
    sequences.append(seq)
    labels.append(activity_type)
    subjects.append(subject)

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Subject-based train-test split
unique_subjects = np.unique(subjects)
train_subjects = unique_subjects[:6]
test_subjects = unique_subjects[6:]
print(f"Training subjects: {train_subjects}")
print(f"Testing subjects: {test_subjects}")

train_idx = [i for i, s in enumerate(subjects) if s in train_subjects]
test_idx = [i for i, s in enumerate(subjects) if s in test_subjects]

train_sequences = [sequences[i] for i in train_idx]
train_labels = [labels_encoded[i] for i in train_idx]
train_subjects_list = [subjects[i] for i in train_idx]
test_sequences = [sequences[i] for i in test_idx]
test_labels = [labels_encoded[i] for i in test_idx]
test_subjects_list = [subjects[i] for i in test_idx]

# Custom Dataset
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels, subjects):
        self.sequences = sequences
        self.labels = labels
        self.subjects = subjects
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.subjects[idx]

# Collate function to pad sequences
def collate_fn(batch):
    sequences, labels, subjects = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    max_len = max(lengths)
    padded_sequences = []
    for seq in sequences:
        pad_width = ((0, max_len - len(seq)), (0, 0))
        padded_seq = np.pad(seq, pad_width, mode='constant', constant_values=0)
        padded_sequences.append(padded_seq)
    return (torch.tensor(padded_sequences, dtype=torch.float32),
            torch.tensor(labels, dtype=torch.long),
            torch.tensor(lengths, dtype=torch.long),
            subjects)

# LSTM Model
class ActivityLSTM(nn.Module):
    def __init__(self, input_size=3, hidden_size=64, num_layers=2, num_classes=10, bidirectional=True):
        super(ActivityLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)
    
    def forward(self, x, lengths):
        lstm_out, _ = self.lstm(x)
        batch_size = x.size(0)
        indices = torch.arange(batch_size, device=x.device)
        last_outputs = lstm_out[indices, lengths - 1, :]  # Last non-padded output
        logits = self.fc(last_outputs)
        return logits

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
    
    def __call__(self, test_f1, model):
        score = test_f1
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ActivityLSTM(num_classes=len(le.classes_)).to(device)

# Class weights
class_counts = Counter(train_labels)
class_weights = [1.0 / class_counts.get(i, 1) for i in range(len(le.classes_))]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoaders
train_dataset = SequenceDataset(train_sequences, train_labels, train_subjects_list)
test_dataset = SequenceDataset(test_sequences, test_labels, test_subjects_list)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Training and Evaluation
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        padded_sequences, labels, lengths, _ = [b.to(device) if isinstance(b, torch.Tensor) else b for b in batch]
        optimizer.zero_grad()
        logits = model(padded_sequences, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            padded_sequences, labels, lengths, _ = [b.to(device) if isinstance(b, torch.Tensor) else b for b in batch]
            logits = model(padded_sequences, lengths)
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='weighted')
    return acc, f1

# Training Loop
early_stopping = EarlyStopping(patience=5)
for epoch in range(50):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    test_acc, test_f1 = evaluate(model, test_loader, device)
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Test Acc={test_acc:.4f}, Test F1={test_f1:.4f}")
    early_stopping(test_f1, model)
    

Training subjects: ['U1' 'U2' 'U21' 'U22' 'U3' 'U4']
Testing subjects: ['U5' 'U6' 'U7']
Epoch 1: Loss=2.3154, Test Acc=0.1500, Test F1=0.0429
Epoch 2: Loss=2.3113, Test Acc=0.1000, Test F1=0.0316
Epoch 3: Loss=2.2917, Test Acc=0.1000, Test F1=0.0316
Epoch 4: Loss=2.2893, Test Acc=0.1000, Test F1=0.0333
Epoch 5: Loss=2.2786, Test Acc=0.1000, Test F1=0.0429
Epoch 6: Loss=2.2860, Test Acc=0.1000, Test F1=0.0500
Epoch 7: Loss=2.2523, Test Acc=0.1000, Test F1=0.0857
Epoch 8: Loss=2.2429, Test Acc=0.0500, Test F1=0.0750
Epoch 9: Loss=2.2149, Test Acc=0.0500, Test F1=0.0750
Epoch 10: Loss=2.2172, Test Acc=0.0000, Test F1=0.0000
Epoch 11: Loss=2.1834, Test Acc=0.0000, Test F1=0.0000
Epoch 12: Loss=2.1464, Test Acc=0.0000, Test F1=0.0000
Epoch 13: Loss=2.0275, Test Acc=0.0000, Test F1=0.0000
Epoch 14: Loss=2.1310, Test Acc=0.0000, Test F1=0.0000
Epoch 15: Loss=2.0614, Test Acc=0.0000, Test F1=0.0000
Epoch 16: Loss=2.1163, Test Acc=0.0500, Test F1=0.0286
Epoch 17: Loss=1.8605, Test Acc=0.0000, T

Counter({9: 7, 1: 5, 3: 5, 5: 5, 4: 4, 2: 4, 6: 4, 7: 2, 8: 2})